# Analytic order of Sha

Exercise 1. Write a function which takes as input an elliptic curve over Q, and outputs the analytic order of sha.

In [ ]:
def analytic_sha_order(E):

    # TODO

Exercise 2. Run your function on several of the elliptic curves in the LMFDB using the API, and verify your result with the order given there.

Exercise 3. Can you extend the function to arbitrary number fields?